### Основные типы агентов:
- Tool Calling_Agent - базовый тип агента, который можно кастомизировать
- ReAct (Reasoning and Acting) - агенты составляющие план и работающие по циклу, пока не добьются результата.
Работют с инструментами, принимающими один параметр на вход
- Structured chat - могут работать с инструментами на много параметров на входе
- Self Ask With Search - реализация Self-Ask, работают только с инструментами поиска
- XMLAgent, JSONAgent - для работы с чат моделями в форматах
- OpenAI Tools Agents

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI
import os
from getpass import getpass

# course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса:")

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

### Tool calling Agent

Для создания агента понадобится:
- специальный промпт
- инструменты
- модель
- специальные методы

In [7]:
# Загрузим уже знакомые нам инструменты
from langchain.agents import load_tools, tool

tools = load_tools(["llm-math"], llm=llm)

@tool
def get_word_length(word: str) -> int:
    """Возвращает длину слова

    Parameters
    ----------
    word
        Слово

    Returns
    -------
        Количество букв
    """
    return len(word)


tools.append(get_word_length) #Добавляем свой тул

In [8]:
from langchain_core.prompts import ChatPromptTemplate

#Составляем специальный промпт
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты полезный ассистент")
    ,("human", "{input}")
    ,("placeholder", "{agent_scratchpad}")
])

In [9]:
# создаём агента, передадим модель, инструменты и промпт
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm,tools, prompt)

In [ ]:
# Для запуска агента используется класс AgentExecutor
agent_executor = AgentExecutor(agent=agent
                               ,tools=tools
                               ,max_iterations=5    #Максимальной количество итераций (дефолт 15)
                               ,handle_parsing_errors = True    #Агент не упадет с ошибкой, если не удастся распарсить ответ
                               ,verbose=True    #будет показывать промежуточные шаги
                               )

agent_executor.invoke({"input": "Сколько букв в слове зачёт?"})

#Видим в результате, что агент вызвал наш созданный инструмент и использовал результат



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'зачёт'}`


5В слове "зачёт" 5 букв.

> Finished chain.


{'input': 'Сколько букв в слове зачёт?', 'output': 'В слове "зачёт" 5 букв.'}

### Добавляем Tool calling agent память

In [11]:
prompt = ChatPromptTemplate.from_messages([
        ("system", "Ты полезный ассистент"),
        ("placeholder", "{chat_history}"), # <-- история диалога
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
])

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#Создадим словарь с историями по session_id
store = {}

#Функция, возвращающая историю диалога по session ID
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [13]:
agent_with_history = RunnableWithMessageHistory(agent_executor
                                                ,get_session_history
                                                ,input_messages_key="input"     #Название переменной запроса
                                                ,history_messages_key="chat_history"    #Название переменной для истории из шаблона
                                                )

agent_with_history.invoke({"input": "Сколько букв в слове зачёт?"}
                          ,config={"configurable": {"session_id": "1"}})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'зачёт'}`


5В слове "зачёт" 5 букв.

> Finished chain.


{'input': 'Сколько букв в слове зачёт?',
 'chat_history': [],
 'output': 'В слове "зачёт" 5 букв.'}

In [ ]:
agent_with_history.invoke({"input": "А в слове ёж?"},
                            config={"configurable": {"session_id": "1"}})

#Здесь можно увидеть что появилась история запросов



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'ёж'}`


2В слове "ёж" 2 буквы.

> Finished chain.


{'input': 'А в слове ёж?',
 'chat_history': [HumanMessage(content='Сколько букв в слове зачёт?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='В слове "зачёт" 5 букв.', additional_kwargs={}, response_metadata={})],
 'output': 'В слове "ёж" 2 буквы.'}

### ReAct Agent

Состоит из нескольких подходов:
- Reasoning (Техника Chain of Thoughts)
- Acting (Acting Plan Generator - генерация плана действий)

Многие шаблоны промптов хранятся в Langchain hub. Можно их брать и модернизировать под себя - https://smith.langchain.com/hub

In [16]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
print(prompt)
print(prompt.template)

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question:

### ReAct и использование двух тулов:
- serpapi (поиск в гугле) (100 запросов бесплатно)
- llm-math
  Попробуем задать модели задачку загуглить температуру в Сочи и возвести её в квадрат

In [ ]:
# Добавляем полученный ключ в переменную среды
os.environ["SERPAPI_API_KEY"] = getpass(prompt="Введите ваш ключ SerpApi:")

import langchain
from langchain.agents import create_react_agent

tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Явно задается используемый промпт
prompt = hub.pull("hwchase17/react") #Забираем промпт
# prompt = hub.pull("hwchase17/react-chat")     #Тоже самое, но с историей

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent,
                               tools=tools, 
                               verbose=True,
                               max_iterations=15, # максимальное количество итераций агента (15 по умолчанию)
                               handle_parsing_errors=True)

agent_executor.invoke({"input": "Температура в Сочи? Ответь в Цельсиях и возведи в квадрат"})

In [ ]:
#Добавим новый тул, вычисляющий площадь треугольника
@tool
def compute_triangle_area(a, b, c):
    """Вычисляет площадь треугольника по длинам его сторон"""
    s = (a + b + c) / 2
    return (s * (s - a) * (s - b) * (s - c)) ** 0.5

# Добавим его к списку тулзов
tools.append(compute_triangle_area)

#Инициализируем агента
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#Здесь получим ошибку, так как для 3х аргументов нужен уже Structured Chat Agent
agent_executor.invoke({"input": "Посчитай площадь треугольника со сторонами 2, 2, 2"})

### Structured Chat Agent
Похож на ReAct, но для multi-input tools

In [17]:
# Посмотрим на промпт Structured Chat агента, используемый по умолчанию
prompt = hub.pull("hwchase17/structured-chat-agent")
print(prompt.messages[0].prompt.template)

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation


In [ ]:
from langchain.agents import create_structured_chat_agent

#Добавим новый тул, вычисляющий площадь треугольника
@tool
def compute_triangle_area(a, b, c):
    """Вычисляет площадь треугольника по длинам его сторон"""
    s = (a + b + c) / 2
    return (s * (s - a) * (s - b) * (s - c)) ** 0.5

tools = load_tools([ "llm-math"], llm=llm)
tools.append(compute_triangle_area)

agent = create_structured_chat_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Посчитай площадь треугольника со сторонами 2, 2, 2"})

#Наш агент задействует наш тул для расчета площади треугольника



> Entering new AgentExecutor chain...
Thought: Для расчета площади треугольника со сторонами 2, 2, 2 я могу использовать формулу Герона или специальный случай для равностороннего треугольника. Так как треугольник равносторонний, я могу использовать известную формулу для его площади.

Action:
```
{
  "action": "compute_triangle_area",
  "action_input": {
    "a": 2,
    "b": 2,
    "c": 2
  }
}
```1.7320508075688772Action:
```
{
  "action": "Final Answer",
  "action_input": "Площадь треугольника со сторонами 2, 2, 2 составляет примерно 1.732."
}
```

> Finished chain.


{'input': 'Посчитай площадь треугольника со сторонами 2, 2, 2',
 'output': 'Площадь треугольника со сторонами 2, 2, 2 составляет примерно 1.732.'}

### Self-Ask
Техника Self-Ask позволяет llm не сразу отвечать на запрос, а сначала делать промежуточные поисковые запросы и, основываясь на результатах, генерировать окончательный ответ
Это дает прирост в качестве при ответах на сложные запросы

Self-Ask with search может использовать только 1 инструмент - любой поисковик

Например, если спросить - Какая самая большая река протекает в городе, в котором родился автор 'Капитанской дочки'?"
Здесь нужно задать два вопроса и найти между ними пересечения, если модель пытается ответить одним запросом - у нее скорее всего не получится



Шаблон промпта
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate answer: New Zealand.
So the final answer is: No

Question: {input}
Are followup questions needed here:{agent_scratchpad}

In [21]:
from langchain.agents import create_self_ask_with_search_agent, Tool
from langchain_community.utilities import SerpAPIWrapper

prompt = hub.pull("hwchase17/self-ask-with-search") #Тянем запрос

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='useful for when you need to answer questions'
    )
]

#Создаем тул  с поисковиком - обязательно называем Intermediate Answer

agent = create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "Какая самая большая река протекает в городе, в котором родился автор романа Капитанская дочка?"})

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ValidationError: 1 validation error for SerpAPIWrapper
  Value error, Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass `serpapi_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

### Agent - RAG

Схема такая:
- создаем агента и в качестве инструмента даем ему ретривер
- ретривер достает релевантную инфу из бд
- бд создадим распарсив страницу пиццерии
- тем самым научим отвечать на вопросы по сайту



In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://allopizza.su/spb/kupchino/about")
data = loader.load()

#Грузим данные

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data) #Сплитуем доки

#Создаем эмбеддинги
embeddings = HuggingFaceBgeEmbeddings(model_name="cointegrated/LaBSE-en-ru"
                                               ,model_kwargs={"device": "cpu"}
                                            #    ,model_kwargs={"device": "cuda"}
                                               )

#Создаем векторную бд
db_embed = FAISS.from_documents(texts, embeddings)

#Создаем ретривера
retriever = db_embed.as_retriever()

D:\Temp\TEMP\ipykernel_10976\550220920.py:10: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="cointegrated/LaBSE-en-ru"


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\n.khabarov\.cache\huggingface\hub\models--cointegrated--LaBSE-en-ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [25]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(retriever
                             ,"search_web" #Название тула
                             ,"Searches and returns data from page" #Описание инструмента
                             )
tools = [tool]

In [27]:
prompt = hub.pull("hwchase17/react") #Качаем шаблон промпта

from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
result = agent_executor.invoke({"input": "Из скольки авто состоит автопарк компании?"})
result["output"]

#Агент умеет отвечать на вопросы по нашему сайту

'Автопарк компании "Алло! Пицца" состоит из более чем 200 автомобилей.'

### SQL и агенты
Langchain имеет инструменты, которые позволяют взаимодействовать с БД без использования SQL запросов пользователем

SQLDatabaseChain - цепочка, выполняющая запросы к базе
SQLDatabaseToolKit - ящик инструментов, который подается в агента, а именно:
    - QuerySQLDataBaseTool - инструмент создания и выполнения запросов
    - InfoSQLDatabaseTool - инструмент для получения инфо о таблицах
    - ListSQLDatabaseTool - названия таблиц
    - QueryChekerTool - инструмент проверяющий запросы на синтаксис
  

In [5]:
from langchain_community.utilities import SQLDatabase

# подключимся к базе
db = SQLDatabase.from_uri("sqlite:///pizzeria.db")
print(db.dialect) #Какой SQL юзаем

sqlite


In [7]:
# получим названия всех таблиц в базе
print(db.get_usable_table_names())

#Получение всех таблиц

['clients', 'orders', 'pizza_orders', 'pizzas']


In [6]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm) #Создаем тулкит

agent_executor = create_sql_agent(llm
                                  ,toolkit=toolkit
                                  ,verbose=True) #Создаем агента

In [ ]:
question = "В каком месяце 2023 года было больше всего заказов?"

resp = agent_executor.invoke({"input": {question}})

#Агент ответил на естественном языке - Final Answer: В июне 2023 года было больше всего заказов (114 заказов).



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ''  clients, orders, pizza_orders, pizzasThe most relevant table to look into for the question about the month with the most orders in 2023 is the "orders" table. I'll now check the schema of the "orders" table to see the relevant columns, particularly ones related to dates or order counts.

Action: sql_db_schema  
Action Input: orders  
CREATE TABLE orders (
	id INTEGER NOT NULL, 
	order_date DATE, 
	client_id INTEGER, 
	quantity INTEGER, 
	price NUMERIC(10, 2), 
	PRIMARY KEY (id), 
	FOREIGN KEY(client_id) REFERENCES clients (id)
)

/*
3 rows from orders table:
id	order_date	client_id	quantity	price
1	2023-05-01	1	1	750.00
2	2023-05-01	2	4	12000.00
3	2023-05-01	1	2	2750.00
*/To determine the month in 2023 with the highest number of orders, I will need to count the orders grouped by month from the "order_date" field, and filter for the year 2023. Then, I can order the results by the count of orders 

In [ ]:
db.run('''
SELECT strftime('%m', order_date) AS month, COUNT(*) AS total_orders 
FROM orders 
WHERE strftime('%Y', order_date) = '2023' 
GROUP BY month 
ORDER BY total_orders DESC
LIMIT 1
''')

#Проверка - всё верно

"[('06', 114)]"

In [15]:
question = "Опиши таблицу clients"

resp = agent_executor.invoke({"input": {question}})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ''  clients, orders, pizza_orders, pizzasThe "clients" table exists in the database. I will now query its schema to describe its structure. 

Action: sql_db_schema  
Action Input: clients  
CREATE TABLE clients (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	address VARCHAR, 
	phone VARCHAR, 
	PRIMARY KEY (id)
)

/*
3 rows from clients table:
id	name	age	address	phone
1	Виссарион Огарев	26	ул. Ордынка М. 1108	+7-943-980-0050
2	Гордей Брагин	38	ул. Коммунистическая 847	+7-928-427-6017
3	Лукьян Артёмов	26	ул. Лапина 1121	+7-956-078-2824
*/I now know the final answer. 
Final Answer: The "clients" table has the following structure:
- id (INTEGER, PRIMARY KEY)
- name (VARCHAR)
- age (INTEGER)
- address (VARCHAR)
- phone (VARCHAR)

Sample data from the "clients" table includes:
1. Виссарион Огарев, 26, ул. Ордынка М. 1108, +7-943-980-0050
2. Гордей Брагин, 38, ул. Коммунистическая 847, +7-928-427

In [ ]:
question = "В какой день недели пиццы наименьшего размера приносят наибольшую выручку?"

resp = agent_executor.invoke({"input": {question}})

#Final Answer: Понедельник (Monday) имеет наибольшую выручку от пицц наименьшего размера.



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  clients, orders, pizza_orders, pizzasI need to check the schema of the relevant tables that may contain information about pizza sizes, their sales, and the days of the week on which they are sold. The likely relevant tables are `orders`, `pizza_orders`, and `pizzas`. I'll start by getting the schema for these tables. 

Action: sql_db_schema  
Action Input: "orders, pizza_orders, pizzas"  
CREATE TABLE orders (
	id INTEGER NOT NULL, 
	order_date DATE, 
	client_id INTEGER, 
	quantity INTEGER, 
	price NUMERIC(10, 2), 
	PRIMARY KEY (id), 
	FOREIGN KEY(client_id) REFERENCES clients (id)
)

/*
3 rows from orders table:
id	order_date	client_id	quantity	price
1	2023-05-01	1	1	750.00
2	2023-05-01	2	4	12000.00
3	2023-05-01	1	2	2750.00
*/


CREATE TABLE pizza_orders (
	order_id INTEGER NOT NULL, 
	pizza_id INTEGER NOT NULL, 
	PRIMARY KEY (order_id, pizza_id), 
	FOREIGN KEY(order_id) REFERENCES orders (id),

In [7]:
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

# создаем пайтон агента
agent_model = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# создаем sql агента
agent_executor_sql = create_sql_agent(llm, db=db, verbose=True)

In [8]:
from langchain.agents import Tool
# оба агента станут инструментами для верхнеуровнего агента
tools_mix=[
        Tool(
            name="PythonAgent",
            func=agent_model.run,
            description="""Useful to run python commands""",
        ),
        Tool(
            name="SQLAgent",
            func=agent_executor_sql.run,
            description="""Useful to query sql tables""",
        ),
    ]

In [13]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

# верхнеуровневый агент с промптом ReAct
prompt = hub.pull("hwchase17/react")
agent_mix = create_react_agent(
    tools=tools_mix,
    llm=llm,
    prompt=prompt,
) 

d:\Khabarov\Репозиторий\llm_practice_course\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
agent_executor = AgentExecutor(agent=agent_mix, tools=tools_mix, verbose=True,
                              handle_parsing_errors=True)

#Создаем непосредственно финального агента

In [ ]:
question = "Возьми из таблицы orders данные о количестве заказов в день. И построй график для первых 10 дней."
agent_executor.invoke({'input':question})

#Раз через раз можешь сам создать график



> Entering new AgentExecutor chain...
Чтобы ответить на вопрос, сначала необходимо получить данные о количестве заказов в день из таблицы `orders`. После получения данных нужно будет отобразить их в виде графика для первых 10 дней.

Action: SQLAgent  
Action Input: "SELECT order_date, COUNT(*) as order_count FROM orders GROUP BY order_date ORDER BY order_date LIMIT 10;"  


> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  clients, orders, pizza_orders, pizzasThe relevant table for the query is "orders." I will check the schema of the "orders" table to confirm the columns available for querying. 

Action: sql_db_schema  
Action Input: "orders"  
CREATE TABLE orders (
	id INTEGER NOT NULL, 
	order_date DATE, 
	client_id INTEGER, 
	quantity INTEGER, 
	price NUMERIC(10, 2), 
	PRIMARY KEY (id), 
	FOREIGN KEY(client_id) REFERENCES clients (id)
)

/*
3 rows from orders table:
id	order_date	client_id	quantity	price
1	2023-05-01	1	1	750.00
2	2023-05-01

Python REPL can execute arbitrary code. Use with caution.


To answer the question, I need to execute the provided code, which creates a plot showing the number of orders over specific days. The code uses libraries `matplotlib` and `pandas` to generate and display a line graph based on the given data. I will run the code to see if it executes correctly and produces the desired output.

Action: Python_REPL  
Action Input: ```python
import matplotlib.pyplot as plt
import pandas as pd

# Данные о количестве заказов
data = {
    'order_date': ['2023-05-01', '2023-05-02', '2023-05-03', '2023-05-06', 
                   '2023-05-07', '2023-05-08', '2023-05-09', '2023-05-10', 
                   '2023-05-13', '2023-05-14'],
    'order_count': [6, 4, 1, 3, 6, 6, 5, 2, 3, 6]
}

# Создание DataFrame
df = pd.DataFrame(data)
df['order_date'] = pd.to_datetime(df['order_date'])

# Построение графика
plt.figure(figsize=(10, 5))
plt.plot(df['order_date'], df['order_count'], marker='o')
plt.title('Количество заказов по дням')
plt.xlabel('Дата')
plt.ylabel('Коли

{'input': 'Возьми из таблицы orders данные о количестве заказов в день. И построй график для первых 10 дней.',
 'output': 'Я извлек следующие данные о количестве заказов в первый 10 дней:\n\n- 2023-05-01: 6 заказов\n- 2023-05-02: 4 заказа\n- 2023-05-03: 1 заказ\n- 2023-05-06: 3 заказа\n- 2023-05-07: 6 заказов\n- 2023-05-08: 6 заказов\n- 2023-05-09: 5 заказов\n- 2023-05-10: 2 заказа\n- 2023-05-13: 3 заказа\n- 2023-05-14: 6 заказов\n\nЕсли вам нужен график, вам придется выполнить предоставленный код на вашей локальной машине или в среде, где доступна библиотека для построения графиков, например, matplotlib.'}

### LangServe - сервис для деплоя Langchain-цепей, агентов и любых runnable объектов как Rest_API сервисов

LangServe интегрирован с FastAPI и для валидации использует pydantic

В файле server.py создадим код, который будет слушать запросы по адресу (адрес в файле)
В нем два эндпоита, один для вопросов о пиццерии, второй шутит шутки на заданную тему

In [3]:
# Проверяем "шуточную" цепочку
import requests

response = requests.post(
    "http://127.0.0.1:8000/joke/invoke",
    json={'input': {'topic': 'robot'}}
)
response.json()

{'output': {'content': 'Why did the robot go on a diet?\n\nBecause it had too many bytes!',
  'additional_kwargs': {'refusal': None},
  'response_metadata': {'token_usage': {'completion_tokens': 16,
    'prompt_tokens': 13,
    'total_tokens': 29,
    'completion_tokens_details': {'accepted_prediction_tokens': 0,
     'audio_tokens': 0,
     'reasoning_tokens': 0,
     'rejected_prediction_tokens': 0},
    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
   'model_name': 'gpt-4o-mini-2024-07-18',
   'system_fingerprint': 'fp_34a54ae93c',
   'id': 'chatcmpl-ByaIjg1Lh5uy2hjk6wOP9CpEtZ8wy',
   'service_tier': 'default',
   'finish_reason': 'stop',
   'logprobs': None},
  'type': 'ai',
  'name': None,
  'id': 'run--79ae2b52-68ea-470c-b867-3dbc3003ccf6-0',
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': [],
  'usage_metadata': {'input_tokens': 13,
   'output_tokens': 16,
   'total_tokens': 29,
   'input_token_details': {'audio': 0, 'cache_read': 0},
   'out

In [5]:
# Проверяем RAG-агента
response = requests.post(
    "http://127.0.0.1:8000/rag_agent/invoke",
    json={'input': {'input':'Из скольки авто состоит автопарк компании?'}}
)
response.json()

{'output': 'Автопарк компании "Алло! Пицца" состоит из более 200 легковых автомобилей.',
 'metadata': {'run_id': 'cf3cf189-ed07-41ac-a249-45f713338176',
  'feedback_tokens': []}}